In [75]:
#from 
import json
from dataclasses import dataclass
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from utils import load_simple_json,load_json
from sklearn.metrics import recall_score,precision_score,confusion_matrix,accuracy_score

TEST data

In [83]:
TEST_DATA = load_json("../pert_large/enough/0522_03_fix/test.jsonl")
len(TEST_DATA)

931

In [84]:
preds = load_simple_json("../pert_large/claim/0522_03_w_noise/test_fix.json")#test_ens4_p74

In [85]:
def join_pred_data(pred_dic,data):
    re_lst=[]
    data_map = {i["id"]:idx for idx,i in enumerate(data)}
    for key in pred_dic:
        buf=None
        if int(key) in data_map.keys():
            buf = data[data_map[int(key)]]
            buf["predict_label"]=pred_dic[key]['predict_label']
            re_lst.append(buf)
    return re_lst

In [86]:
pair_data = join_pred_data(preds,TEST_DATA)
len(pair_data)

931

In [201]:
def score(in_data):
    data = in_data.copy()
    data =[i for i in data if i["label"]!="NOT ENOUGH INFO"]
    label_map = {'supports':0,'refutes':1}
    gt_label,pred_label=[],[]
    for i in data:
        if i["label"]== "NOT ENOUGH INFO":
            continue
        gt_label.append(label_map[i["label"]])
        pred_label.append(i["predict_label"])
    recall,precision = recall_score(gt_label,pred_label),precision_score(gt_label,pred_label)
    print("accuracy: ",accuracy_score(gt_label,pred_label))
    print("precision: {}, recall: {}, f1: {}".format(recall,precision,(2*precision*recall)/(precision+recall)))
    print(confusion_matrix(gt_label,pred_label))

In [88]:
score(pair_data)

accuracy:  0.8183060109289617
precision: 0.7269503546099291, recall: 0.7854406130268199, f1: 0.7550644567219152
[[394  56]
 [ 77 205]]


In [49]:
pair_data[0]

{'id': 5208,
 'claim': '光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。',
 'predicted_pages_1': ['顯微鏡', '光學顯微鏡'],
 'predicted_evidence': [['顯微鏡', 0],
  ['光學顯微鏡', 0],
  ['顯微鏡', 6],
  ['顯微鏡', 1],
  ['光學顯微鏡', 3]],
 'enough': 1,
 'predict_label': 1}

In [40]:
with open("../pert_large/enough/0522_03/test.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(enough_data):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

## kfold 

In [264]:
def join_kf_dic(dics,thresh=0.5):#majority voting
    re_dic={}
    for dic in dics:
        for key in dic:
            if key in re_dic:
                re_dic[key]+=[dic[key]["prob"]]
            else:
                re_dic[key]=[dic[key]["prob"]]
    for key in re_dic:
        if thresh:
            re_dic[key]=sum(np.array(re_dic[key]))/len(dics)
        else:
            re_dic[key] = int(np.median(re_dic[key]))
    return re_dic

In [311]:
TEST_DATA = load_json("../pert_large/enough/0522_nclu_neg_no_pid_folds/all_test_clu_page_sent_nclu_e01_ens5_prob.jsonl")
#TEST_DATA = load_json("../pert_large/enough/0522_nclu_neg_no_pid_folds/test_ens5.jsonl")
#preds = load_simple_json("../pert_large/claim/0522_03_w_noise_folds/test/test_f1.json")
len(TEST_DATA)

8597

In [287]:
def pred_thresh(dic,thresh):
    for k in dic:
        if dic[k]['predict_label']>thresh:
            dic[k]['predict_label']=1
        else:
            dic[k]['predict_label']=0
    return dic

In [313]:
processed_preds=[]
processed_dicts=[]
#＃preds = ["test_f0","test_f1","test_f2","test_f3","test_f4"]
preds = ["all_test_f0","all_test_f1","all_test_f2","all_test_f3","all_test_f4"]
pred_dics =[]
thr_param =[]
for pred_path in preds:
    pred_dics.append(load_simple_json("../pert_large/claim/0522_clu_w_noise_npid_folds/all_test/"+pred_path+".json"))
pred_dic = join_kf_dic(pred_dics)
pred_dic = {key:{"predict_label":pred_dic[key]} for key in pred_dic}
pred_dic = pred_thresh(pred_dic,0.29)
pair_data = join_pred_data(pred_dic,TEST_DATA)
#score(pair_data)

In [314]:
len(pair_data)

8597

In [283]:
pred_dic["21498"]

{'predict_label': 0}

## post preocess for adding back removed data

In [320]:
label_map = {0:'supports',1:'refutes'}
for i in pair_data:
    i["predicted_label"]=label_map[i["predict_label"]]
    if i["enough"]<0.1:
        i["predicted_label"]="NOT ENOUGH INFO"
pair_data[0]

{'id': 21498,
 'claim': '雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與人類關係密切。',
 'predicted_pages_1': ['雞形目'],
 'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]],
 'enough': 0.9632043480873108,
 'predict_label': 0,
 'predicted_label': 'supports'}

In [321]:
ori_data = load_json("../preprocess/all_test.jsonl")
len(ori_data)

9038

In [322]:
pair_data[0]

{'id': 21498,
 'claim': '雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與人類關係密切。',
 'predicted_pages_1': ['雞形目'],
 'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]],
 'enough': 0.9632043480873108,
 'predict_label': 0,
 'predicted_label': 'supports'}

In [323]:
mapping_idx = {d["id"]:idx for idx,d in enumerate(pair_data)}
re_lst = []
for d in ori_data:
    if d["id"] in mapping_idx.keys():
        data = pair_data[mapping_idx[d["id"]]]
        re_lst.append({"id":data["id"],"predicted_label":data["predicted_label"],"predicted_evidence":data["predicted_evidence"]})
    else:
        re_lst.append({"id":d["id"],"predicted_label":"NOT ENOUGH INFO","predicted_evidence":None})

In [324]:
len(re_lst)

9038

In [327]:
with open("../submissions/all_test_clu_page_sent_claim029_e005_ens5.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(re_lst):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

In [326]:
re_lst[:5]

[{'id': 21498,
  'predicted_label': 'supports',
  'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]]},
 {'id': 13037,
  'predicted_label': 'refutes',
  'predicted_evidence': [['新約書信', 5],
   ['大公書信', 0],
   ['新約書信', 3],
   ['保羅_(使徒)', 3],
   ['大公書信', 10]]},
 {'id': 18652,
  'predicted_label': 'supports',
  'predicted_evidence': [['羅伯·昆蘭', 4], ['羅伯·昆蘭', 0], ['羅伯·昆蘭', 1]]},
 {'id': 21378,
  'predicted_label': 'supports',
  'predicted_evidence': [['2015年美國網球公開賽女子單打比賽', 0],
   ['2016年法國網球公開賽', 8],
   ['2015年溫布頓網球錦標賽', 9],
   ['2015年美國網球公開賽女子單打比賽', 3],
   ['2003年美國網球公開賽女子單打比賽', 0]]},
 {'id': 18624,
  'predicted_label': 'supports',
  'predicted_evidence': [['漢光武帝', 4],
   ['漢光武帝', 0],
   ['漢光武帝', 10],
   ['漢光武帝', 6],
   ['漢光武帝', 9]]}]

### for cluster enough

In [235]:
len(pair_data)

8597

In [268]:
with open("../postprocess/all_test_clu_page_sent_claim_nclu_ens5_probs.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(pair_data):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")